In [94]:
import numpy as np
import pandas as pd
import math

In [95]:
# reading data from csvs

courses_df = pd.read_csv('courses.csv')

sections_df = pd.read_csv('sections.csv')

professors_df = pd.read_csv('professors.csv')

rooms_df = pd.read_csv('rooms.csv')

time_slots_df = pd.read_csv('timeslots.csv')

In [96]:
# generating all possible combination of courses and sections
courses_list = np.array(sections_df[["Course", "Section"]])

In [97]:
def generate_courses():
    courses_list = np.array(sections_df[["Course", "Section"]])
    
    courses = []
    
    for i in range(len(courses_list)):
        course = courses_list[i][0]
        section = courses_list[i][1]
        coursetype = np.array(courses_df[courses_df["Course"] == course]["Type"])[0]
        strength = np.array(sections_df[(sections_df["Course"] == course) & (sections_df["Section"] == section)]["Strength"])[0]
        
        currcourse = np.array([course, coursetype, section, strength])
        
        courses.append(currcourse)
        
    return np.array(courses)

def generate_slots():
    
    days = len(np.array(time_slots_df["Day"].drop_duplicates()))
    
    times = np.array((time_slots_df[["Start Time", "End Time"]].apply(lambda x: '-'.join(x),axis=1)).drop_duplicates())
    
    slots = []
    
    for i in range(days):
            slots.append(times)
            
    return np.array(slots)
    
def generate_rooms():
    
    rooms = np.array(rooms_df[['Room', 'Capacity']])
    
    return rooms

def generate_professors():
    professors = np.array(professors_df[['Professor', "Course"]])
    
    return professors


In [98]:
def evaluate_slot(slot, coursetype):
    #res = True
    
    if(coursetype == "0"):
        pos_ones = []

        for i in range(len(slot) - 1, -1, -1):
            if(slot[i] == '1'):
                pos_ones.append(len(slot) - i - 1)

        # check if if the same day or not

        if((math.floor(pos_ones[0] / 6)) == (math.floor(pos_ones[1] / 6)) ): #if same then give penalty
            return False

        days = []

        # Iterate over the string in steps of 6
        for i in range(0, len(slot), 6):
            days.append(slot[i:i+6])
        #check if slots in adjacent days or not
        for i in range(len(days) - 1):
            if( ('1' in days[i]) and ('1' in days[i+1]) ):
                return False
    
    else: #for lab
        pos_ones = []

        for i in range(len(slot) - 1, -1, -1):
            if(slot[i] == '1'):
                pos_ones.append(len(slot) - i - 1)
                
        # check if if the same day or not

        if((math.floor(pos_ones[0] / 6)) != (math.floor(pos_ones[1] / 6)) ): #if not same then give penalty
            return False
        
        if(np.abs(pos_ones[1] - pos_ones[0]) != 1 ):
            return False

    return True


def generate_random_slot(coursetype):
    
    
    while(True):
        slotsmask = format(0, '030b')  
        binary_list = list(slotsmask)
        indexes_to_set = [np.random.randint(0,30), np.random.randint(0,30)]
        while indexes_to_set[0] == indexes_to_set[1]:
            indexes_to_set[1] = np.random.randint(0, 30)
        for idx in indexes_to_set:
            binary_list[idx] = '1'
        slotbits = ''.join(binary_list)
        
        if(evaluate_slot(slotbits, coursetype) == True):
            return slotbits
    

def generate_individual(courses, professors, rooms, slots):
    # the choromosome will binary encoded with the below format
    # [8bits for course][8bits for room][30 bits mask for slots]
    # since there will be 6*5 = 30 slots in a week
    
    numRooms = len(rooms)
    
    individual = []
    
    
    for i in range(len(courses)):
        index = i
        coursebits = format(index, '08b')
        
        coursetype = courses[i][1]
        
        typebits = ""
        
        if(coursetype == "Theory"):
            typebits += '0'
        else:
            typebits += '1'
        
        randroom = np.random.randint(0, numRooms)
        roombits = format(randroom, '08b')
        
        courseprofs = [index for index, prof in enumerate(professors) if prof[1] == courses[i][0]]
        currprofindex = np.random.choice(courseprofs)
        profbits = format(currprofindex, '08b')

        slotbits = generate_random_slot(typebits)
        
        chromosome = coursebits + typebits + roombits + profbits + slotbits
        
        individual.append(chromosome)
        
    return np.array(individual)
    

In [99]:
def generate_population(popsize):
    
    courses = generate_courses()
    slots = generate_slots()
    rooms = generate_rooms()
    professors = generate_professors()
    
    population = []
    
    for i in range(popsize):
        population.append(generate_individual(courses, professors, rooms, slots))
        
    return np.array(population)

In [100]:
def evaluate_individual(individual, realCourses, realRooms):
    
    courses = []
    rooms = []
    professors = []
    timeslots = []
    
    
    totalpenalty = 0
    
    for i in range(len(individual)):
        
        #each chromosome
        
        course = individual[i][:8]
        coursetype = individual[i][8:9]
        room = individual[i][9:17]
        prof = individual[i][17:25]
        slot = individual[i][25:]
            

        courses.append(course)
        rooms.append(room)
        professors.append(prof)
        timeslots.append(slot)
        
    
        
    for i in range(len(rooms)):
        currRoom = rooms[i]
        
        for j in range(i+1, len(rooms)):
            if(rooms[j] == currRoom):
                # perfrom bitwise AND of corresponding timeslots to check clashes
                timeslot1 = timeslots[i]
                timeslot2 = timeslots[j]
                result = int(timeslot1, 2) & int(timeslot2, 2)
                if(result > 0):
                    totalpenalty += 5
    
    for i in range(len(professors)):
        currProf = professors[i]
        
        for j in range(i+1, len(professors)):
            if(professors[j] == currProf):
                # perfrom bitwise AND of corresponding timeslots to check clashes
                timeslot1 = timeslots[i]
                timeslot2 = timeslots[j]
                result = int(timeslot1, 2) & int(timeslot2, 2)
                if(result > 0):
                    totalpenalty += 5
                    
    for i in range(len(courses)):
        currCourse = int(courses[i], 2)
        
        courseStrength = int(realCourses[currCourse][3])
        
        currRoom = int(rooms[i], 2)
        
        roomCapacity = int(realRooms[currRoom][1])
        
        if(courseStrength > roomCapacity):
            totalpenalty += 5
            
            
    #check if 1 section assigned 2 different courses in same timeslot
    
    for i in range(len(timeslots)):
        currtimeslot = timeslots[i]
        
        for j in range(i+1, len(timeslots)):
            result = int(currtimeslot, 2) & int(timeslots[j], 2)
            
            if(result > 0):
                index1 = int(courses[i], 2)
                index2 = int(courses[j], 2)
                
                section1 = realCourses[index1][2]
                section2 = realCourses[index2][2]
                
                if(section1 == section2):
                    totalpenalty += 5
    
    for i in range(len(professors)):
        currProf = professors[i]
        
        profcount = 1
        
        for j in range(i+1, len(professors)):
            if(professors[j] == currProf):
                profcount += 1
                
        if(profcount > 3):
            totalpenalty += 5
                    
                    
                    
                    
    fitness = 1 / (1 + totalpenalty) # fine tune
    return fitness


def gen_fitnesses(population, courses, rooms):
    
    fitnesses = []
    
    for i in range(len(population)):
        currindividual = population[i]
        currfitness = evaluate_individual(currindividual, courses, rooms)
        fitnesses.append(currfitness)
        
    return np.array(fitnesses)
        

In [101]:
def roulette_wheel_selection(population, fitness_values, num_parents):
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected_parents = []
    selected_indices = set()  # To keep track of selected parent indices
    while len(selected_parents) < num_parents:
        r = np.random.rand()
        cumulative_prob = 0
        for i, prob in enumerate(probabilities):
            cumulative_prob += prob
            if r <= cumulative_prob and i not in selected_indices:
                selected_parents.append(population[i])
                selected_indices.add(i)
                break
                
    
    return np.array(selected_parents)

In [102]:
def crossover(parent1, parent2):
    crossover_point = np.random.randint(0,3)
    
    crossover_point = (crossover_point + 1)*8

    offspring1 = []
    offspring2 = []
    
    for i in range(len(parent1)):
        chromosome1 = parent1[i]
        chromosome2 = parent2[i]
        
        resChromo1 = chromosome1[:crossover_point] + chromosome2[crossover_point:]
        resChromo2 = chromosome2[:crossover_point] + chromosome1[crossover_point:]
        
        offspring1.append(resChromo1)
        offspring2.append(resChromo2)
        
    offspring1 = np.array(offspring1)
    offspring2 = np.array(offspring2)
    
    return np.array([offspring1, offspring2])
    

In [103]:
def mutateroom(room, rooms):
    lenRooms = len(rooms)
    
    numRoom = int(room, 2)
    
    vals = [-1,1]
    
    i = np.random.randint(0, 2)
    
    while( (numRoom + vals[i]) < 0 or (numRoom + vals[i]) == lenRooms):
        i = np.random.randint(0, 2)
        
    numRoom += vals[i]
    
    room = format(numRoom, '08b')
    
    return room
    
    
def mutateprof(prof, professors):
    
    numProf = int(prof, 2)
    
    currProf = professors[numProf]
    
    course = currProf[1]
    
    newProf = np.random.choice(np.where(professors[:, 1] == course)[0])
    
    prof = format(newProf, '08b')
    
    return prof
      

def mutation(individual, rooms, professors, probability):
    
    newindividual = []
    
    for i in range(len(individual)):
        #each chromosome
        
        course = individual[i][:8]
        coursetype = individual[i][8:9]
        room = individual[i][9:17]
        prof = individual[i][17:25]
        slot = individual[i][25:]

        if(np.random.rand() < probability):
            
            pos = np.random.randint(0,3)
            
            if(pos == 0):
                room = mutateroom(room, rooms)
            elif(pos == 1):
                prof = mutateprof(prof,professors)
            elif(pos == 2):
                slot = generate_random_slot(coursetype)
                
                
        newchromosome = course+coursetype+room+prof+slot
     
        newindividual.append(newchromosome)
        
        
    return np.array(newindividual)

In [104]:
def cullpopulation(population,courses, rooms, cullcount):
    fitnesses = gen_fitnesses(population,courses,rooms)
    
    # Sort population and fitnesses based on fitness values in descending order
    sorted_indices = np.argsort(fitnesses)[::-1]  # Reverse the sorted indices
    population = population[sorted_indices]
    fitnesses = fitnesses[sorted_indices]
    
    # Remove the least fit individuals
    population = population[:-cullcount]
    
    return population

In [105]:
# def decode_individual(individual, courses, slots, rooms, professors):
    
#     days_lookup = np.array(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])
    
#     for i in range(len(individual)):
#         # current chromosome 
#         course = int(individual[i][:8], 2)
#         coursetype = int(individual[i][8:9],2)
#         room = int(individual[i][9:17],2)
#         prof = int(individual[i][17:25],2)
#         slot = individual[i][25:]
        
#         print(courses[course])
            
#         print(rooms[room])
#         print(professors[prof])
             
#         days = []
        
#         slot = slot[::-1]

#         # Iterate over the string in steps of 6
#         for j in range(0, len(slot), 6):
#             days.append(slot[j:j+6])
        
#         for j in range(len(days)):
#             if('1' in days[j]):
#                 print(days_lookup[j])
#                 indices = [k for k, char in enumerate(days[j]) if char == '1']
#                 #print(indices)
#                 for l in range(len(indices)):
#                     print(slots[j][indices[l]])
                
#         print("\n\n")        
        

    
def decode_individual(individual, courses, slots, rooms, professors):
    
    days_lookup = np.array(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])
    slots_lookup = slots[0] 
    
    table = pd.DataFrame(index=days_lookup, columns=slots_lookup)
    
    for day in days_lookup:
        for time_slot in slots_lookup:
            table.loc[day, time_slot] = ''
    
    
    for i in range(len(individual)):
        # current chromosome 
        course = int(individual[i][:8], 2)
        coursetype = int(individual[i][8:9],2)
        room = int(individual[i][9:17],2)
        prof = int(individual[i][17:25],2)
        slot = individual[i][25:]
        
#         print(courses[course])
        
        data = courses[course][0] + " " + courses[course][2] + " " + rooms[room][0] + ", "
            
        #print(rooms[room])
        #print(professors[prof])
             
        days = []
        
        slot = slot[::-1]

        # Iterate over the string in steps of 6
        for j in range(0, len(slot), 6):
            days.append(slot[j:j+6])
        
        for j in range(len(days)):
            if('1' in days[j]):
                #print(days_lookup[j])
                indices = [k for k, char in enumerate(days[j]) if char == '1']
                #print(indices)
                for l in range(len(indices)):
                    #print(slots[j][indices[l]])
                    table.loc[days_lookup[j], slots[j][indices[l]]] += data
                
        #print("\n\n") 
        
    return table
        
              

In [112]:
courses = generate_courses()
slots = generate_slots()
rooms = generate_rooms()
professors = generate_professors()

population = generate_population(10)

iterations = 0

while(iterations < 1000000):
    iterations += 1
    fitnesses = gen_fitnesses(population,courses, rooms)
    if(1 in fitnesses):       
        break
    
    parents = roulette_wheel_selection(population, fitnesses, 2)
    children = crossover(parents[0],parents[1])
    
    children[0] = mutation(children[0],rooms,professors,0.3)
    children[1] = mutation(children[1],rooms,professors,0.3)
    
    population = np.vstack([population, children])
    
    population = cullpopulation(population,courses, rooms, 2)
    
    #print(iterations)
    

fitnesses = gen_fitnesses(population,courses, rooms)

index = -1

for i in range(len(fitnesses)):
    if(fitnesses[i] == 1):
        index = i
        break
        
individual = population[index]

print("NUM ITERATIONS " + str(iterations))

table = decode_individual(individual,courses,slots,rooms,professors)

styled_table = table.style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', 'lightblue'), ('color', 'black')]},
    {'selector': 'td', 'props': [('border', '1px solid grey'), ('text-align', 'center')]}
])

# Display the styled table
display(styled_table)

NUM ITERATIONS 5045


,8:30 AM-9:50 AM,10:05 AM-11:25 AM,11:40 AM-1:00 PM,1:15 PM-2:35 PM,2:50 PM-4:10 PM,4:25 PM-5:45 PM
Monday,"Calculus E Lab 19, Programming Fundamentals B Lab 15,",,"Digital Logic B Lab 4,","Physics B A107, Digital Logic C Lab 16, Digital Logic Lab E A111,","Programming Fundamentals A A203, Digital Logic D A207, Digital Logic Lab E A111,","Physics C A203, Physics D A209, Digital Logic E Lab 5,"
Tuesday,"Programming Fundamentals D Lab 3, Programming Fundamentals Lab B Lab 13,","Programming Fundamentals Lab B Lab 13, Digital Logic Lab C A101,","Calculus A A102, Digital Logic Lab C A101,","Physics A A201,","Calculus B A109, Calculus D A108, Programming Fundamentals Lab C A211,","Programming Fundamentals E Lab 1, Digital Logic A Lab 7, Programming Fundamentals Lab C A211,"
Wednesday,"Digital Logic D A207,","Calculus C Lab 19, Digital Logic Lab A Lab 14,","Programming Fundamentals Lab D A102, Digital Logic Lab A Lab 14,","Physics E Lab 1, Programming Fundamentals Lab D A102,","Programming Fundamentals Lab E Lab 15, Digital Logic Lab B A201,","Programming Fundamentals C A204, Programming Fundamentals Lab E Lab 15, Digital Logic Lab B A201,"
Thursday,"Calculus D A108, Programming Fundamentals Lab A A103,","Calculus E Lab 19, Digital Logic C Lab 16, Programming Fundamentals Lab A A103,","Physics D A209, Digital Logic A Lab 7,","Physics B A107,","Physics C A203, Programming Fundamentals B Lab 15,","Digital Logic B Lab 4,"
Friday,"Calculus C Lab 19, Digital Logic Lab D A102,","Programming Fundamentals C A204, Digital Logic Lab D A102,","Physics A A201, Physics E Lab 1,","Calculus A A102, Calculus B A109, Programming Fundamentals D Lab 3, Programming Fundamentals E Lab 1,","Programming Fundamentals A A203,","Digital Logic E Lab 5,"
